<a href="https://colab.research.google.com/github/basmadokkar/Data-vs-Model-Sparsity-/blob/main/Network_sparsity_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff
import glob
import os
import cv2
import tensorflow as tf
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda,Dense, Activation, MaxPool2D, Concatenate, Flatten
from keras import optimizers
from keras.optimizers import Adam
from keras.metrics import MeanIoU
from keras import backend as K
from sklearn import metrics
from google.colab import drive
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from skimage.filters import threshold_otsu, threshold_multiotsu
from cv2 import adaptiveThreshold
from PIL import Image
import keras


In [ ]:
def get_num_neurons_conv_layer(layer):
    kernel_depth = layer.filters
    kernel_width = 3
    num_neurons = kernel_depth*kernel_width*kernel_width
    return num_neurons

def count_possible_connections(model):
    total_connections = 0
    num_layers = len(model.layers)

    for layer1 in model.layers:
      try:
        if 'conv' in layer1.get_config()['name'] :
          num_neurons1 = get_num_neurons_conv_layer(layer1)
          for layer2 in model.layers:
                  if 'conv' in layer2.get_config()['name']:
                      num_neurons2 = get_num_neurons_conv_layer(layer2)
                      total_connections += num_neurons1 * num_neurons2
      except:
        continue

    return total_connections // 2


In [ ]:
def count_connections(model):
    total_connections = 0
    input_shape = model.layers[0].input_shape[0][1:]

    for layer in model.layers:
        try:

            output_shape = layer.output_shape[0][1:]
        except:
            output_shape = layer.output_shape[1:]

        if isinstance(layer, keras.layers.Dense):
            num_connections = input_shape[0] * output_shape[0]

        elif isinstance(layer, keras.layers.Conv2D):
            num_connections = layer.kernel_size[0] * layer.kernel_size[1] * input_shape[-1] * output_shape[-1]


        else:
            num_connections = 0

        total_connections += num_connections
        input_shape = output_shape

    return total_connections



## First Network : UNet

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS =  3
def conv_block(inputs, filter_count, pool=True, batchnorm = True):


    #first layer
    x = Conv2D(filter_count, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    #second layer
    x = Conv2D(filter_count, 3, padding = 'same', kernel_initializer = 'he_normal')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x



def deconv_block(inputs, concat_layer, filter_count, pool = False):
    u = UpSampling2D((2,2), interpolation='bilinear')(inputs)
    c = Concatenate()([u, concat_layer])
    x = conv_block(c, filter_count, pool = pool, batchnorm = True)
    return u, c, x

def make_me_a_unet(shape, num_classes):

    inputs = Input(shape) # 768 x 1152

    # Downsampling side of the UNET i.e. the encoder !

    x1, p1 = conv_block(inputs, 32, pool=True, batchnorm=True)
    x2, p2 = conv_block(p1, 64, pool=True, batchnorm=True)
    x3, p3 = conv_block(p2, 96, pool=True, batchnorm=True)
    x4, p4 = conv_block(p3, 128, pool=True, batchnorm=True)
    b = conv_block(p4, 256, pool=False, batchnorm=True)

    # Upsampling side of the UNET i.e the decoder !

    u1, c1, x5 = deconv_block(b, x4, 128)
    u2, c2, x6 = deconv_block(x5, x3, 96)
    u3, c3, x7 = deconv_block(x6, x2, 64)
    u4, c4, x8 = deconv_block(x7, x1, 32)

    # The output layer

    output = Conv2D(num_classes, 1, padding='same', activation='sigmoid')(x8)

    #softmax for multiclass classification, num_classes = 23 !

    return Model(inputs, output)



model = make_me_a_unet((IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS), 1)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 32)         896       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 32)         0         ['batch_normalization[0][0

In [ ]:

num_possible_connections = count_possible_connections(model)
print("Total number of possible connections in the model:", num_possible_connections)
num_connections = count_connections(model)
print("Total number of connections in the model:", num_connections)

Total number of possible connections in the model: 130201384
Total number of connections in the model: 2378624


In [ ]:
2378624/130201384

0.01826880734232441

## Second Network = DenseUNet

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import DenseNet121


def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_densenet121_unet(input_shape):
    """ Input """
    inputs = Input(input_shape, name="input_1")

    """ Pre-trained DenseNet121 Model """
    densenet = DenseNet121(include_top=False, weights=None, input_tensor=inputs)

    """ Encoder """

    s1 = densenet.get_layer("input_1").output       ## 512
    s2 = densenet.get_layer("conv1/relu").output    ## 256
    s3 = densenet.get_layer("pool2_relu").output ## 128
    s4 = densenet.get_layer("pool3_relu").output  ## 64

    """ Bridge """
    b1 = densenet.get_layer("pool4_relu").output  ## 32

    """ Decoder """
    d1 = decoder_block(b1, s4, 256)             ## 64
    d2 = decoder_block(d1, s3, 128)             ## 128
    d3 = decoder_block(d2, s2, 64)             ## 256
    d4 = decoder_block(d3, s1, 32)              ## 512

    """ Outputs """
    outputs = Conv2D(3, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs)
    return model

# def build_densenet121_unet(input_shape):
#     """ Input """
#     inputs = Input(input_shape)

#     """ Pre-trained DenseNet121 Model """
#     densenet = DenseNet121(include_top=False, weights=None, input_tensor=inputs)

#     """ Encoder """
#     s1 = inputs                                       ## 256
#     s2 = densenet.get_layer("conv1/relu").output    ## 256
#     s3 = densenet.get_layer("pool2_relu").output ## 128
#     s4 = densenet.get_layer("pool3_relu").output  ## 64

#     """ Bridge """
#     b1 = densenet.get_layer("pool4_relu").output  ## 32

#     """ Decoder """
#     d1 = decoder_block(b1, s4, 256)             ## 64
#     d2 = decoder_block(d1, s3, 128)             ## 128
#     d3 = decoder_block(d2, s2, 64)             ## 256
#     d4 = decoder_block(d3, s1, 32)              ## 512

#     """ Outputs """
#     outputs = Conv2D(3, 1, padding="same", activation="sigmoid")(d4)

#     model = Model(inputs, outputs)
#     return model



if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = build_densenet121_unet(input_shape)
    model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 262, 262, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 128, 128, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1/conv[0][0]']    

In [ ]:
model.layers[3]

In [ ]:

num_possible_connections = count_possible_connections(model)
print("Total number of possible connections in the model:", num_possible_connections)
num_connections = count_connections(model)
print("Total number of connections in the model:", num_connections)


Total number of possible connections in the model: 22319712
Total number of connections in the model: 8515712


In [ ]:
8515712/22319712

0.3815332384217144

## Third Network = DenseNet

In [ ]:
densenet = DenseNet121(include_top=False,input_shape=(256,256,3))
num_possible_connections = count_possible_connections(densenet)
print("Total number of possible connections in the model:", num_possible_connections)
num_connections = count_connections(densenet)
print("Total number of connections in the model:", num_connections)

29084464/29084464 [==============================] - 0s 0us/step
Total number of possible connections in the model: 26542080
Total number of connections in the model: 6870208


In [ ]:
6870208/26542080

0.25884211033950616

## Fourth Network : ConvNet

In [ ]:
# output
IMG_HEIGHT =  128
IMG_WIDTH  = 128
IMG_CHANNELS = 1
kernel_initializer =  'he_uniform'
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(s)
    c1 = Dropout(0.1)(c1)

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    # c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    # c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(96, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
    # c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    # c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)

    flat = Flatten()(c5)
    outputs = Dense(100,activation='softmax')(flat)
    model = Model(inputs=[inputs], outputs=[outputs])


    return model


def get_model():
  return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

model = get_model()
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_78 (Conv2D)          (None, 128, 128, 16)      160       
                                                                 
 dropout_17 (Dropout)        (None, 128, 128, 16)      0         
                                                                 
 conv2d_79 (Conv2D)          (None, 128, 128, 16)      2320      
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 64, 64, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_80 (Conv2D)          (None, 64, 64, 32)        4640      
                                                           

In [ ]:
num_possible_connections = count_possible_connections(model)
print("Total number of possible connections in the model:", num_possible_connections)
num_connections = count_connections(model)
print("Total number of connections in the model:", num_connections)

Total number of possible connections in the model: 37324800
Total number of connections in the model: 1122192


In [ ]:
 1122192 / 37324800

0.030065586419753085